> ### Sentiment Analysis

>> #### DataSet

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from IPython.display import clear_output

datasets, info = tfds.load(name="imdb_reviews", with_info=True, as_supervised=True)
train_size = info.splits['train'].num_examples
clear_output()

In [2]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<PAD>"), y_batch

In [3]:
# Build Vocabulary
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [4]:
vocabulary.most_common()[:3]

[(b'<PAD>', 214309), (b'the', 61137), (b'a', 38564)]

In [5]:
# Truncate vocabulary
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [6]:
# Preprocessing word
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [7]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

In [8]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

In [9]:
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [10]:
# Create model and train
embed_size = 128
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                                mask_zero=True, input_shape=[None]),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5


2023-01-23 16:08:24.129127: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT8
    }
  }
}

	while inferring type of node 'cond_41/output/_22'
2023-01-23 16:08:25.245648: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-01-23 16:08:26.850078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-23 16:08:27.007324: I tensorflow/compiler/xla/service

  1/782 [..............................] - ETA: 1:31:09 - loss: 0.6925 - accuracy: 0.5625

2023-01-23 16:08:27.329812: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 [==============================] - 28s 26ms/step - loss: 0.5389 - accuracy: 0.7207
Epoch 2/5
782/782 [==============================] - 13s 17ms/step - loss: 0.3422 - accuracy: 0.8568
Epoch 3/5
782/782 [==============================] - 13s 17ms/step - loss: 0.1904 - accuracy: 0.9316
Epoch 4/5
782/782 [==============================] - 12s 16ms/step - loss: 0.1331 - accuracy: 0.9519
Epoch 5/5
782/782 [==============================] - 14s 18ms/step - loss: 0.1200 - accuracy: 0.9558
